In [1]:
import pandas as pd
import numpy as np
import os
import random
from collections import defaultdict
from itertools import combinations

# ========= CONFIG =========
INDEX_CSV = r"D:\5th sem\mini_project\dataset\socofing_index_features.csv"
OUT_DIR = r"D:\5th sem\mini_project\dataset\pairs"

TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15
RANDOM_SEED = 42

os.makedirs(OUT_DIR, exist_ok=True)
rng = random.Random(RANDOM_SEED)

# ========= STEP 1: SUBJECT-LEVEL SPLIT =========
def split_subjects(df):
    subjects = sorted(df["subject_id"].unique().tolist())
    rng.shuffle(subjects)

    n = len(subjects)
    n_train = int(n * TRAIN_RATIO)
    n_val = int(n * VAL_RATIO)
    n_test = n - n_train - n_val

    subj_train = set(subjects[:n_train])
    subj_val = set(subjects[n_train:n_train + n_val])
    subj_test = set(subjects[n_train + n_val:])

    print(f"Total subjects: {n}")
    print(f"Train: {len(subj_train)}, Val: {len(subj_val)}, Test: {len(subj_test)}")

    df_train = df[df["subject_id"].isin(subj_train)].copy()
    df_val = df[df["subject_id"].isin(subj_val)].copy()
    df_test = df[df["subject_id"].isin(subj_test)].copy()

    return df_train, df_val, df_test

# ========= STEP 2: POSITIVE PAIRS =========
def build_positive_pairs(df_split):
    """
    Build all positive pairs within each (subject_id, finger) group.
    Returns a list of dicts with agg_path_1, agg_path_2, label=1, etc.
    """
    pos_pairs = []

    # group by subject_id + finger
    grouped = df_split.groupby(["subject_id", "finger"])
    for (subj, finger), group in grouped:
        # if only one sample for this finger, skip
        if len(group) < 2:
            continue

        # create all combinations of rows within this group
        indices = list(group.index)
        for i, j in combinations(indices, 2):
            r1 = df_split.loc[i]
            r2 = df_split.loc[j]

            pos_pairs.append({
                "agg_path_1": r1["agg_path"],
                "agg_path_2": r2["agg_path"],
                "label": 1,
                "subject_id_1": r1["subject_id"],
                "subject_id_2": r2["subject_id"],
                "finger_1": r1["finger"],
                "finger_2": r2["finger"],
                "alter_level_1": r1["alter_level"],
                "alter_level_2": r2["alter_level"],
            })

    print(f"Positive pairs: {len(pos_pairs)}")
    return pos_pairs

# ========= STEP 3: NEGATIVE PAIRS =========
def build_negative_pairs(df_split, num_pos_pairs, max_neg_factor=1.2):
    """
    Build random negative (impostor) pairs.
    Aim for roughly num_pos_pairs negatives (limited by max_neg_factor * num_pos_pairs).
    """
    neg_pairs = []

    # index by subject_id
    subj_to_indices = defaultdict(list)
    for idx, row in df_split.iterrows():
        subj_to_indices[row["subject_id"]].append(idx)

    subjects = list(subj_to_indices.keys())
    total_neg_target = int(min(len(df_split)**2, num_pos_pairs * max_neg_factor))

    attempts = 0
    max_attempts = total_neg_target * 10  # safety

    while len(neg_pairs) < total_neg_target and attempts < max_attempts:
        attempts += 1

        subj1, subj2 = rng.sample(subjects, 2)
        idx1 = rng.choice(subj_to_indices[subj1])
        idx2 = rng.choice(subj_to_indices[subj2])

        r1 = df_split.loc[idx1]
        r2 = df_split.loc[idx2]

        neg_pairs.append({
            "agg_path_1": r1["agg_path"],
            "agg_path_2": r2["agg_path"],
            "label": 0,
            "subject_id_1": r1["subject_id"],
            "subject_id_2": r2["subject_id"],
            "finger_1": r1["finger"],
            "finger_2": r2["finger"],
            "alter_level_1": r1["alter_level"],
            "alter_level_2": r2["alter_level"],
        })

    print(f"Negative pairs: {len(neg_pairs)} (target ~{num_pos_pairs})")
    return neg_pairs

# ========= MAIN GENERATION =========
def generate_pairs_for_split(df_split, split_name):
    print(f"\n=== Building pairs for {split_name} ===")
    pos_pairs = build_positive_pairs(df_split)
    neg_pairs = build_negative_pairs(df_split, num_pos_pairs=len(pos_pairs))
    all_pairs = pos_pairs + neg_pairs
    rng.shuffle(all_pairs)

    df_pairs = pd.DataFrame(all_pairs)
    out_path = os.path.join(OUT_DIR, f"pairs_{split_name}.csv")
    df_pairs.to_csv(out_path, index=False)
    print(f"Saved {len(df_pairs)} pairs to {out_path}")

def main():
    df = pd.read_csv(INDEX_CSV)
    print(f"Loaded index with {len(df)} rows.")

    df_train, df_val, df_test = split_subjects(df)

    generate_pairs_for_split(df_train, "train")
    generate_pairs_for_split(df_val, "val")
    generate_pairs_for_split(df_test, "test")

if __name__ == "__main__":
    main()


Loaded index with 110540 rows.
Total subjects: 600
Train: 420, Val: 90, Test: 90

=== Building pairs for train ===
Positive pairs: 1404452
Negative pairs: 1685342 (target ~1404452)
Saved 3089794 pairs to D:\5th sem\mini_project\dataset\pairs\pairs_train.csv

=== Building pairs for val ===
Positive pairs: 291419
Negative pairs: 349702 (target ~291419)
Saved 641121 pairs to D:\5th sem\mini_project\dataset\pairs\pairs_val.csv

=== Building pairs for test ===
Positive pairs: 302579
Negative pairs: 363094 (target ~302579)
Saved 665673 pairs to D:\5th sem\mini_project\dataset\pairs\pairs_test.csv


Columns: agg_path_1, agg_path_2, label, subject_id_1, subject_id_2, finger_1, finger_2, alter_level_1, alter_level_2.

Each row is one pair; label = 1 for positive (same subject+finger), 0 for negative.

In [3]:
import os
import pandas as pd

OUT_DIR = r"D:\5th sem\mini_project\dataset\pairs"

for split in ["train", "val", "test"]:
    csv_path = os.path.join(OUT_DIR, f"pairs_{split}.csv")
    assert os.path.exists(csv_path), f"Missing: {csv_path}"
    df = pd.read_csv(csv_path)
    print(f"{split}: rows={len(df)}, positives={(df['label']==1).sum()}, negatives={(df['label']==0).sum()}")
    print(df.head(3), "\n")


train: rows=3089794, positives=1404452, negatives=1685342
                                          agg_path_1  \
0  D:\5th sem\mini_project\dataset\NPYSF\Altered_...   
1  D:\5th sem\mini_project\dataset\NPYSF\Altered_...   
2  D:\5th sem\mini_project\dataset\NPYSF\Altered_...   

                                          agg_path_2  label  subject_id_1  \
0  D:\5th sem\mini_project\dataset\NPYSF\Altered_...      1           303   
1  D:\5th sem\mini_project\dataset\NPYSF\Altered_...      1           347   
2  D:\5th sem\mini_project\dataset\NPYSF\Real_npy...      0           132   

   subject_id_2       finger_1       finger_2 alter_level_1 alter_level_2  
0           303   index_finger   index_finger        medium          hard  
1           347  middle_finger  middle_finger        medium          hard  
2           258   thumb_finger  little_finger          easy          none   

val: rows=641121, positives=291419, negatives=349702
                                          agg_pat

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class FingerprintPairsDataset(Dataset):
    def __init__(self, pairs_csv):
        self.df = pd.read_csv(pairs_csv)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load aggregated SIFT vectors
        v1 = np.load(row["agg_path_1"]).astype(np.float32)
        v2 = np.load(row["agg_path_2"]).astype(np.float32)

        # Ensure shape [128] -> [1, 128] or [128] as needed
        x1 = torch.from_numpy(v1)  # [128]
        x2 = torch.from_numpy(v2)  # [128]

        label = torch.tensor(row["label"], dtype=torch.float32)  # 0 or 1

        return x1, x2, label

# Example: create loaders
if __name__ == "__main__":
    train_csv = r"D:\5th sem\mini_project\dataset\pairs\pairs_train.csv"
    val_csv = r"D:\5th sem\mini_project\dataset\pairs\pairs_val.csv"

    train_dataset = FingerprintPairsDataset(train_csv)
    val_dataset = FingerprintPairsDataset(val_csv)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

    print(f"Train pairs: {len(train_dataset)}, Val pairs: {len(val_dataset)}")


Train pairs: 3089794, Val pairs: 641121


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===== Siamese branch: MLP on 128-D vector =====
class SiameseBranch(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embedding_dim),
        )

    def forward(self, x):
        # x: [batch, 128]
        return self.net(x)

class SiameseNetwork(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.branch = SiameseBranch(input_dim, embedding_dim)

    def forward(self, x1, x2):
        z1 = self.branch(x1)
        z2 = self.branch(x2)
        return z1, z2

# ===== Contrastive loss =====
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, z1, z2, labels):
        # z1, z2: [batch, emb_dim]
        # labels: [batch] with 1 for positive, 0 for negative
        distances = F.pairwise_distance(z1, z2)  # [batch]
        # standard contrastive loss: positives -> small dist, negatives -> > margin
        loss_pos = labels * torch.pow(distances, 2)
        loss_neg = (1 - labels) * torch.pow(torch.clamp(self.margin - distances, min=0.0), 2)
        loss = torch.mean(loss_pos + loss_neg)
        return loss


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ========== CONFIG ==========
TRAIN_CSV = r"D:\5th sem\mini_project\dataset\pairs\pairs_train.csv"
VAL_CSV   = r"D:\5th sem\mini_project\dataset\pairs\pairs_val.csv"

MODEL_DIR          = r"D:\5th sem\mini_project\dataset\models"
FINAL_MODEL_PATH   = os.path.join(MODEL_DIR, "snn_sift_agg_last.pt")
BEST_MODEL_PATH    = os.path.join(MODEL_DIR, "snn_sift_agg_best.pt")

BATCH_SIZE = 256
LR         = 1e-3
EPOCHS     = 30
MARGIN     = 1.0
PATIENCE   = 5  # early stopping patience

os.makedirs(MODEL_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# ========== DATASET ==========
class FingerprintPairsDataset(Dataset):
    def __init__(self, pairs_csv):
        self.df = pd.read_csv(pairs_csv)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        v1 = np.load(row["agg_path_1"]).astype(np.float32)
        v2 = np.load(row["agg_path_2"]).astype(np.float32)

        x1 = torch.from_numpy(v1)   # [128]
        x2 = torch.from_numpy(v2)   # [128]
        label = torch.tensor(row["label"], dtype=torch.float32)  # 0 or 1

        return x1, x2, label


# ========== MODEL ==========
class SiameseBranch(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embedding_dim),
        )

    def forward(self, x):
        return self.net(x)


class SiameseNetwork(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.branch = SiameseBranch(input_dim, embedding_dim)

    def forward(self, x1, x2):
        z1 = self.branch(x1)
        z2 = self.branch(x2)
        return z1, z2


class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, z1, z2, labels):
        # z1, z2: [batch, emb_dim]
        # labels: [batch] in {0,1}
        distances = F.pairwise_distance(z1, z2)  # [batch]
        loss_pos = labels * torch.pow(distances, 2)
        loss_neg = (1 - labels) * torch.pow(torch.clamp(self.margin - distances, min=0.0), 2)
        loss = torch.mean(loss_pos + loss_neg)
        return loss


# ========== PREPARE DATA ==========
train_dataset = FingerprintPairsDataset(TRAIN_CSV)
val_dataset   = FingerprintPairsDataset(VAL_CSV)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"Train pairs: {len(train_dataset)}, Val pairs: {len(val_dataset)}")


# ========== INIT MODEL / LOSS / OPTIMIZER ==========
model = SiameseNetwork(input_dim=128, embedding_dim=128).to(device)
criterion = ContrastiveLoss(margin=MARGIN)
optimizer = Adam(model.parameters(), lr=LR)


# ========== TRAINING LOOP WITH EARLY STOPPING ==========
best_val_loss = float("inf")
patience_cnt  = 0

for epoch in range(EPOCHS):
    # ---- TRAIN ----
    model.train()
    running_loss = 0.0

    for x1, x2, y in train_loader:
        x1 = x1.to(device)
        x2 = x2.to(device)
        y  = y.to(device)

        optimizer.zero_grad()
        z1, z2 = model(x1, x2)
        loss = criterion(z1, z2, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x1.size(0)

    avg_train_loss = running_loss / len(train_dataset)

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x1, x2, y in val_loader:
            x1 = x1.to(device)
            x2 = x2.to(device)
            y  = y.to(device)

            z1, z2 = model(x1, x2)
            loss = criterion(z1, z2, y)
            val_loss += loss.item() * x1.size(0)

    avg_val_loss = val_loss / len(val_dataset)

    print(f"Epoch {epoch+1}/{EPOCHS} - "
          f"train loss: {avg_train_loss:.4f}, val loss: {avg_val_loss:.4f}")

    # ---- EARLY STOPPING CHECK ----
    if avg_val_loss < best_val_loss - 1e-4:
        best_val_loss = avg_val_loss
        patience_cnt = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"  New best model saved with val loss {best_val_loss:.4f}")
    else:
        patience_cnt += 1
        print(f"  No val improvement. Patience {patience_cnt}/{PATIENCE}")
        if patience_cnt >= PATIENCE:
            print("Early stopping triggered.")
            break

# Save last-epoch model as well (optional)
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"Training complete. Best val loss: {best_val_loss:.4f}")
print(f"Best model saved to: {BEST_MODEL_PATH}")
print(f"Last model saved to: {FINAL_MODEL_PATH}")


Using device: cpu
Train pairs: 3089794, Val pairs: 641121


In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ========== CONFIG ==========
TRAIN_CSV = r"D:\5th sem\mini_project\dataset\pairs\pairs_train.csv"
VAL_CSV   = r"D:\5th sem\mini_project\dataset\pairs\pairs_val.csv"

MODEL_DIR          = r"D:\5th sem\mini_project\models"
FINAL_MODEL_PATH   = os.path.join(MODEL_DIR, "snn_sift_agg_last.pt")
BEST_MODEL_PATH    = os.path.join(MODEL_DIR, "snn_sift_agg_best.pt")

BATCH_SIZE = 256          # try 256; if GPU has room, 512 can be faster
LR         = 1e-3
EPOCHS     = 30
MARGIN     = 1.0
PATIENCE   = 5            # early stopping

os.makedirs(MODEL_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# ========== DATASET WITH CACHED FEATURES ==========
class CachedFingerprintPairsDataset(Dataset):
    """
    Loads all unique aggregated SIFT vectors into RAM once,
    then each pair just indexes into this feature matrix.
    """
    def __init__(self, pairs_csv):
        df = pd.read_csv(pairs_csv)
        self.n_pairs = len(df)

        # 1) Collect all unique paths used in this split
        all_paths = pd.concat([df["agg_path_1"], df["agg_path_2"]]).unique()
        print(f"[{pairs_csv}] unique feature files: {len(all_paths)}")

        # 2) Build mapping path -> index
        self.path_to_idx = {p: i for i, p in enumerate(all_paths)}
        self.idx_to_path = list(all_paths)  # optional, for debugging

        # 3) Load all features ONCE into memory
        feats = []
        for p in all_paths:
            v = np.load(p).astype(np.float32)  # [128]
            feats.append(v)
        feats = np.stack(feats, axis=0)       # [N_images, 128]
        self.features = torch.from_numpy(feats)  # float32 tensor

        # 4) Convert pair paths to integer indices
        self.idx1 = df["agg_path_1"].map(self.path_to_idx).to_numpy(dtype=np.int64)
        self.idx2 = df["agg_path_2"].map(self.path_to_idx).to_numpy(dtype=np.int64)

        # 5) Labels
        self.labels = df["label"].to_numpy(dtype=np.float32)

    def __len__(self):
        return self.n_pairs

    def __getitem__(self, idx):
        i1 = self.idx1[idx]
        i2 = self.idx2[idx]
        y  = self.labels[idx]

        x1 = self.features[i1]  # [128]
        x2 = self.features[i2]  # [128]

        # Clone to avoid potential in-place ops on shared tensor
        return x1.clone(), x2.clone(), torch.tensor(y, dtype=torch.float32)


# ========== MODEL ==========
class SiameseBranch(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embedding_dim),
        )

    def forward(self, x):
        return self.net(x)


class SiameseNetwork(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.branch = SiameseBranch(input_dim, embedding_dim)

    def forward(self, x1, x2):
        z1 = self.branch(x1)
        z2 = self.branch(x2)
        return z1, z2


class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, z1, z2, labels):
        distances = F.pairwise_distance(z1, z2)  # [batch]
        loss_pos = labels * torch.pow(distances, 2)
        loss_neg = (1 - labels) * torch.pow(torch.clamp(self.margin - distances, min=0.0), 2)
        loss = torch.mean(loss_pos + loss_neg)
        return loss


# ========== PREPARE DATA ==========
print("Loading training dataset (with caching)...")
train_dataset = CachedFingerprintPairsDataset(TRAIN_CSV)
print("Loading validation dataset (with caching)...")
val_dataset   = CachedFingerprintPairsDataset(VAL_CSV)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,       # start with 0; you can try 2 or 4 later
    pin_memory=True
)
val_loader   = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"Train pairs: {len(train_dataset)}, Val pairs: {len(val_dataset)}")
print(f"Train feature matrix shape: {train_dataset.features.shape}")
print(f"Val feature matrix shape:   {val_dataset.features.shape}")


# ========== INIT MODEL / LOSS / OPTIMIZER ==========
model = SiameseNetwork(input_dim=128, embedding_dim=128).to(device)
criterion = ContrastiveLoss(margin=MARGIN)
optimizer = Adam(model.parameters(), lr=LR)


# ========== TRAINING LOOP WITH EARLY STOPPING ==========
best_val_loss = float("inf")
patience_cnt  = 0

for epoch in range(EPOCHS):
    # ---- TRAIN ----
    model.train()
    running_loss = 0.0

    for x1, x2, y in train_loader:
        x1 = x1.to(device, non_blocking=True)
        x2 = x2.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        optimizer.zero_grad()
        z1, z2 = model(x1, x2)
        loss = criterion(z1, z2, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x1.size(0)

    avg_train_loss = running_loss / len(train_dataset)

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x1, x2, y in val_loader:
            x1 = x1.to(device, non_blocking=True)
            x2 = x2.to(device, non_blocking=True)
            y  = y.to(device, non_blocking=True)

            z1, z2 = model(x1, x2)
            loss = criterion(z1, z2, y)
            val_loss += loss.item() * x1.size(0)

    avg_val_loss = val_loss / len(val_dataset)

    print(f"Epoch {epoch+1}/{EPOCHS} - "
          f"train loss: {avg_train_loss:.4f}, val loss: {avg_val_loss:.4f}")

    # ---- EARLY STOPPING CHECK ----
    if avg_val_loss < best_val_loss - 1e-4:
        best_val_loss = avg_val_loss
        patience_cnt = 0
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"  New best model saved with val loss {best_val_loss:.4f}")
    else:
        patience_cnt += 1
        print(f"  No val improvement. Patience {patience_cnt}/{PATIENCE}")
        if patience_cnt >= PATIENCE:
            print("Early stopping triggered.")
            break

# Save last-epoch model as well (optional)
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"Training complete. Best val loss: {best_val_loss:.4f}")
print(f"Best model saved to: {BEST_MODEL_PATH}")
print(f"Last model saved to: {FINAL_MODEL_PATH}")


Using device: cpu
Loading training dataset (with caching)...
[D:\5th sem\mini_project\dataset\pairs\pairs_train.csv] unique feature files: 38768
Loading validation dataset (with caching)...
[D:\5th sem\mini_project\dataset\pairs\pairs_val.csv] unique feature files: 8171
Train pairs: 3089794, Val pairs: 641121
Train feature matrix shape: torch.Size([38768, 128])
Val feature matrix shape:   torch.Size([8171, 128])


C:\Users\DELL-E7470\anaconda3\envs\hf-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/30 - train loss: 0.1220, val loss: 0.1249
  New best model saved with val loss 0.1249
Epoch 2/30 - train loss: 0.1137, val loss: 0.1239
  New best model saved with val loss 0.1239
Epoch 3/30 - train loss: 0.1112, val loss: 0.1244
  No val improvement. Patience 1/5
Epoch 4/30 - train loss: 0.1097, val loss: 0.1241
  No val improvement. Patience 2/5
Epoch 5/30 - train loss: 0.1085, val loss: 0.1236
  New best model saved with val loss 0.1236
Epoch 6/30 - train loss: 0.1076, val loss: 0.1245
  No val improvement. Patience 1/5
Epoch 7/30 - train loss: 0.1069, val loss: 0.1247
  No val improvement. Patience 2/5
Epoch 8/30 - train loss: 0.1061, val loss: 0.1238
  No val improvement. Patience 3/5
Epoch 9/30 - train loss: 0.1056, val loss: 0.1235
  New best model saved with val loss 0.1235
Epoch 10/30 - train loss: 0.1050, val loss: 0.1233
  New best model saved with val loss 0.1233
Epoch 11/30 - train loss: 0.1045, val loss: 0.1257
  No val improvement. Patience 1/5
Epoch 12/30 - trai

In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# ========= CONFIG =========
VAL_CSV  = r"D:\5th sem\mini_project\dataset\pairs\pairs_val.csv"
TEST_CSV = r"D:\5th sem\mini_project\dataset\pairs\pairs_test.csv"

MODEL_PATH = r"D:\5th sem\mini_project\models\snn_sift_agg_best.pt"

BATCH_SIZE = 512  # can be larger than training since eval has no backprop

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ========= DATASET WITH CACHED FEATURES =========
class CachedFingerprintPairsDataset(Dataset):
    def __init__(self, pairs_csv):
        df = pd.read_csv(pairs_csv)
        self.n_pairs = len(df)

        all_paths = pd.concat([df["agg_path_1"], df["agg_path_2"]]).unique()
        print(f"[{os.path.basename(pairs_csv)}] unique feature files: {len(all_paths)}")

        self.path_to_idx = {p: i for i, p in enumerate(all_paths)}

        feats = []
        for p in all_paths:
            v = np.load(p).astype(np.float32)
            feats.append(v)
        feats = np.stack(feats, axis=0)   # [N_images, 128]
        self.features = torch.from_numpy(feats)

        self.idx1 = df["agg_path_1"].map(self.path_to_idx).to_numpy(dtype=np.int64)
        self.idx2 = df["agg_path_2"].map(self.path_to_idx).to_numpy(dtype=np.int64)
        self.labels = df["label"].to_numpy(dtype=np.float32)

    def __len__(self):
        return self.n_pairs

    def __getitem__(self, idx):
        i1 = self.idx1[idx]
        i2 = self.idx2[idx]
        y  = self.labels[idx]

        x1 = self.features[i1]
        x2 = self.features[i2]

        return x1.clone(), x2.clone(), torch.tensor(y, dtype=torch.float32)

# ========= MODEL (same as training) =========
class SiameseBranch(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embedding_dim),
        )

    def forward(self, x):
        return self.net(x)

class SiameseNetwork(nn.Module):
    def __init__(self, input_dim=128, embedding_dim=128):
        super().__init__()
        self.branch = SiameseBranch(input_dim, embedding_dim)

    def forward(self, x1, x2):
        z1 = self.branch(x1)
        z2 = self.branch(x2)
        return z1, z2

# ========= LOAD MODEL =========
model = SiameseNetwork(input_dim=128, embedding_dim=128).to(device)
state = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state)
model.eval()
print(f"Loaded model from {MODEL_PATH}")

# ========= FUNCTION TO COLLECT DISTANCES AND LABELS =========
def collect_distances_and_labels(pairs_csv):
    dataset = CachedFingerprintPairsDataset(pairs_csv)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0, pin_memory=True)

    all_dist = []
    all_labels = []

    with torch.no_grad():
        for x1, x2, y in loader:
            x1 = x1.to(device, non_blocking=True)
            x2 = x2.to(device, non_blocking=True)
            y  = y.to(device, non_blocking=True)

            z1, z2 = model(x1, x2)
            d = F.pairwise_distance(z1, z2)

            all_dist.append(d.cpu().numpy())
            all_labels.append(y.cpu().numpy())

    all_dist = np.concatenate(all_dist, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    print(f"Collected {len(all_dist)} distances from {os.path.basename(pairs_csv)}")
    return all_dist, all_labels

# ========= EVAL ON VAL: FIND BEST THRESHOLD =========
print("\n--- Collecting distances on VAL set ---")
val_dist, val_labels = collect_distances_and_labels(VAL_CSV)

# Sweep thresholds between min and max distance
t_min, t_max = float(val_dist.min()), float(val_dist.max())
print(f"Val distance range: [{t_min:.4f}, {t_max:.4f}]")

best_acc = 0.0
best_thr = None

thresholds = np.linspace(t_min, t_max, num=200)  # 200 points in range

for thr in thresholds:
    # prediction: match if distance <= thr
    preds = (val_dist <= thr).astype(np.float32)

    correct = (preds == val_labels).sum()
    acc = correct / len(val_labels)

    if acc > best_acc:
        best_acc = acc
        best_thr = thr

print(f"Best threshold on VAL: {best_thr:.4f}, accuracy={best_acc:.4f}")

# Optional: compute TPR/FPR at this threshold
pos_mask = (val_labels == 1)
neg_mask = (val_labels == 0)

tpr = ((val_dist[pos_mask] <= best_thr).sum() / pos_mask.sum())  # genuine accept rate
fpr = ((val_dist[neg_mask] <= best_thr).sum() / neg_mask.sum())  # false accept rate

print(f"VAL TPR (GAR) at best_thr: {tpr:.4f}")
print(f"VAL FPR (FAR) at best_thr: {fpr:.4f}")

# ========= EVAL ON TEST WITH BEST THRESHOLD =========
print("\n--- Collecting distances on TEST set ---")
test_dist, test_labels = collect_distances_and_labels(TEST_CSV)

preds_test = (test_dist <= best_thr).astype(np.float32)
correct_test = (preds_test == test_labels).sum()
acc_test = correct_test / len(test_labels)

pos_mask_t = (test_labels == 1)
neg_mask_t = (test_labels == 0)

tpr_t = ((test_dist[pos_mask_t] <= best_thr).sum() / pos_mask_t.sum())
fpr_t = ((test_dist[neg_mask_t] <= best_thr).sum() / neg_mask_t.sum())

print(f"\nTEST results at best_thr={best_thr:.4f}:")
print(f"  Accuracy: {acc_test:.4f}")
print(f"  TPR (GAR): {tpr_t:.4f}")
print(f"  FPR (FAR): {fpr_t:.4f}")

# ========= OPTIONAL: SAVE DISTANCES FOR PLOTTING LATER =========
OUT_METRICS = r"D:\5th sem\mini_project\models\snn_eval_distances.npz"
np.savez(
    OUT_METRICS,
    val_dist=val_dist,
    val_labels=val_labels,
    test_dist=test_dist,
    test_labels=test_labels,
    best_thr=best_thr
)
print(f"\nSaved distances and labels to {OUT_METRICS}")


Using device: cpu
Loaded model from D:\5th sem\mini_project\models\snn_sift_agg_best.pt

--- Collecting distances on VAL set ---
[pairs_val.csv] unique feature files: 8171


C:\Users\DELL-E7470\anaconda3\envs\hf-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Collected 641121 distances from pairs_val.csv
Val distance range: [0.0000, 1.8863]
Best threshold on VAL: 0.5308, accuracy=0.8350
VAL TPR (GAR) at best_thr: 0.8021
VAL FPR (FAR) at best_thr: 0.1375

--- Collecting distances on TEST set ---
[pairs_test.csv] unique feature files: 8331


C:\Users\DELL-E7470\anaconda3\envs\hf-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Collected 665673 distances from pairs_test.csv

TEST results at best_thr=0.5308:
  Accuracy: 0.8347
  TPR (GAR): 0.7964
  FPR (FAR): 0.1334

Saved distances and labels to D:\5th sem\mini_project\models\snn_eval_distances.npz
